In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_hub_key = os.getenv("LANGCHAIN_HUB_KEY")

In [13]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

In [14]:
llm = ChatOpenAI(openai_api_key=openai_api_key)

In [15]:
graph = Neo4jGraph(
    url="bolt://3.239.79.108:7687",
    username="neo4j",
    password="arrows-sewer-assistants",
)

In [16]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

In [17]:
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [18]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

In [19]:
cypher_chain.invoke({"query": "What is the plot of the movie Toy Story?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Toy Story"})
RETURN m.plot
Full Context:
[{'m.plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room."}]

> Finished chain.


{'query': 'What is the plot of the movie Toy Story?',
 'result': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room."}